In [1]:
import csv
def loadCsv(filename):
    lines = csv.reader(open(filename, "rt"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [2]:
import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [3]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    #print(separated)
    return separated

In [4]:
import math
import numpy as np
def mean(numbers):
    numbers = np.array(numbers)
    
    f1 = numbers[:,0].astype(float)
    f2 = numbers[:,1].astype(float)
    #print(f1)
    #print(f2)
    xx = np.mean(f1)
    yy = np.mean(f2)
    
    cov = np.cov(f1,f2)
    mean = (xx,yy)
    m = list(mean)
    
    return m,cov
    
#def cov(numbers):
    

In [13]:
def summarize(dataset):
    #for x in range(3):
    #print(dataset,'dssd',len(dataset))
    #summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    summaries = []
    #print(dataset,'hye')
    m = mean(dataset)
    #cv = covm(dataset)
    summaries.append(m)
    #summaries.append(cv)
    
    #del summaries[-1]
    #print(summaries)
    return summaries

In [14]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    #print(separated)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    #print(summaries)
    return summaries

In [15]:
import math
import operator
import numpy as np
def calculateProbability(x, mean, cov):
    #pp = np.array(x) - np.array(mean)
    
    #determinant of covariance matrix
    det_cov = np.linalg.det(cov)
    pp = list(map(operator.sub,x,mean)) 
    #print(x,mean)
    #print('covariance',cov)
    #print('x-mean',pp)
    
    
    #inverse of the covariance matrix
    inv_cov = np.linalg.inv(cov)
    
    pt = np.array(pp).reshape(2,1)
    parta = np.dot(pp,inv_cov)
    partb = np.dot(parta,pt)
    part1 = math.exp(-1/2 * partb)
    #print(part1)
    
    part2 = 2 * math.pi * math.sqrt(det_cov)
    #print('part2',part2)
    
    gauss = part1/part2
    
    #print('transpose',pt)
    #inv_cov = np.linalg.det(cov)
    #print(gauss)
    #print(parta)
    #print
    
    return gauss

In [16]:

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    del inputVector [-1]
    #print(inputVector)
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, cov = classSummaries[i]
            #x = inputVector.loc[i][1,2]
            #print(cov)
            probabilities[classValue] *= calculateProbability(inputVector, mean, cov)
            #print(probabilities)
    return probabilities

In [17]:

def predict(summaries, inputVector):
    #print(inputVector)
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [18]:
def getPredictions(parameters, testSet):
    #print(testSet)
    predictions = []
    for i in range(len(testSet)):
        result = predict(parameters, testSet[i])
        predictions.append(result)
    #print('pp',predictions)
    return predictions

In [19]:
def getAccuracy(testSet, predictions):
    correct = 0
    #test = np.array(testSet)
    #test = testSet[-1]
    #print(testSet)
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            correct += 1
            #print(testSet[x][-1])
    return (correct/float(len(testSet))) * 100.0

In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix 
from pandas import * 
def main():
    filename = r'C:\Users\Govind\Downloads\02-Govind\group02\dataset\2d_image_all.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    trainingSet,testSet = splitDataset(dataset, splitRatio)
    print('Size of trainingSet is ',len(trainingSet),'and testing size is ',len(testSet))
    #print(testSet)
    test = []
    for x in range (len(testSet)):
        a = testSet[x][-1]
        test.append(a)
    
    parameters = summarizeByClass(trainingSet)
    #print(parameters)
    #print(testSet)
    
    predictions = getPredictions(parameters, testSet)
    #print(test)
    accuracy = getAccuracy(test, predictions)
    #for x in range (len(test)):
        #print('> predicted=' + repr(predictions) + ', actual=' + repr(testSet[x]))
    
    print('Accuracy:',accuracy)
    print(classification_report(test, predictions))
    results = confusion_matrix(test, predictions)
    print(DataFrame(results, columns=["class-1", "class-2", "class-3"], index=["class-1", "class-2", "class-3"]))
    #print(results)
    
main()

Size of trainingSet is  201 and testing size is  99
Accuracy: 38.38383838383838
             precision    recall  f1-score   support

        1.0       0.12      0.03      0.04        37
        2.0       0.39      0.88      0.54        32
        3.0       0.47      0.30      0.37        30

avg / total       0.32      0.38      0.30        99

         class-1  class-2  class-3
class-1        1       29        7
class-2        1       28        3
class-3        6       15        9
